In [2]:
import os
import random
from tqdm import tqdm
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset

import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split

model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name, max_length=64)

In [18]:
from collections import Counter
SEED = 456

# Load your tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Suppose you have a list of sentences
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '../data')
data = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
dataset_train, dataset_valid = train_test_split(data, test_size=0.3, stratify=data['target'],random_state=SEED)
sentences = dataset_train['text'].tolist()

# Tokenize the sentences
tokenized_sentences = [tokenizer.tokenize(sentence) for sentence in sentences]

# Identify unknown tokens
unknown_tokens = [[token for token in sentence if token == tokenizer.unk_token] for sentence in tokenized_sentences]

# Count the number of unknown tokens in each sentence
unknown_token_counts = [Counter(sentence)[tokenizer.unk_token] for sentence in unknown_tokens]

# Find sentences with mostly unknown tokens
threshold = 4  # Set your own threshold
sentences_with_mostly_unknowns = [sentences[i] for i, count in enumerate(unknown_token_counts) if count > threshold]

In [19]:
for idx, sentence in enumerate(sentences_with_mostly_unknowns):
    print(f"Sentence {idx + 1}: {sentence}")

Sentence 1: 트럼프 돈 찍어 빚 갚겠다…전문가들 세계금융 붕괴 맹공종합
Sentence 2: 朴대통령 추미애號와 대야관계 어떻게 풀어갈까
Sentence 3: 2018 ICT 결산 ③플랫폼 업계 위기 넘으면 또 다른 위기끝
Sentence 4: 베스트셀러 철학은 어떻게 삶의 무기가 되는가 6주째 1위
Sentence 5: 獨 쾰른 성당서 가톨릭 교회 성평등 인간띠 시위
Sentence 6: 세계인과 함께 제주의 봄길을 걷다…유채꽃 국제걷기대회
Sentence 7: 20대 국회 첫 임시회 7일 소집될듯…본회의는 불투명
Sentence 8: 朴대통령 명예퇴진 수용할까…靑 기류변화 없지만 지켜보자
Sentence 9: 정진석 靑 회동서 기명난법 꽐련 문제도 노늬될 껃쏙뽀
Sentence 10: 남부지방에 100㎜ 안팎 장맛비…전국 곳곳 30도 웃도는 더위
Sentence 11: 샤오미 웨어러블시장서 첫 세계 1위…애플·핏비트 꺾었다
Sentence 12: 사이테크 플러스 자동차 바퀴에 깔려도 사는 괴물곤충 외골격의 비밀 풀렸다
Sentence 13: 낟 최고 십꾸도 날씨 따뜨태저…중부 밤늗께 비 조금
Sentence 14: 울산 사흘째 강풍 주의보…강수량 최대 40㎜ 예상
Sentence 15: 현금 만타더니 멷딸 뒤 부도…몬 미들 쭝국 끼어 푀계감사
Sentence 16: 방성윤 섣불렀던 은퇴 죽고 싶었지만 지금도 늦지 않았다면…
Sentence 17: 구쾨 정개트귀 참서칸 최장집 쩐 명예교수
Sentence 18: 한국땅 박찬주 영입 빼제 가닥…영입 깡행 부뉘기 아니다종합뚜보
Sentence 19: 낮 최고기온 23∼31도 초여름 더위…수도권 미세먼지 나쁨
Sentence 20: 주말 N 여행 호남권 달빛 벗 삼아 걸어라…한낮 폭염 피해 즐기는 꿀잼 야행
Sentence 21: 靑 美 북한산 석탄문제 대응 클레임 無…韓정부 신뢰 밝혀
Sentence 22: 러 우주로켓 벼락 맞고도 멀쩡…탑재 위성 정상궤도 올려
Sentence 23: 北당대회 나흘만에 

In [22]:
dataset_test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

test_sentences = dataset_train['text'].tolist()

# Tokenize the sentences
test_tokenized_sentences = [tokenizer.tokenize(sentence) for sentence in test_sentences]

# Identify unknown tokens
test_unknown_tokens = [[token for token in sentence if token == tokenizer.unk_token] for sentence in test_tokenized_sentences]

# Count the number of unknown tokens in each sentence
test_unknown_token_counts = [Counter(sentence)[tokenizer.unk_token] for sentence in test_unknown_tokens]

# Find sentences with mostly unknown tokens
threshold = 1  # Set your own threshold
test_sentences_with_mostly_unknowns = [test_sentences[i] for i, count in enumerate(test_unknown_token_counts) if count > threshold]

In [23]:
for idx, sentence in enumerate(test_sentences_with_mostly_unknowns):
    print(f"Sentence {idx + 1}: {sentence}")

Sentence 1: 삼성의 QLED냐 LG의 올레드냐…내달 영국서 정면승부
Sentence 2: 이녀 년속 씹씅 박쫑훈 선수들·코치님 경녀 덕뿐
Sentence 3: 한국 인터넷 속도 압도적 세계 1위…보급률도 최고
Sentence 4: 러 남부서 또 아파트 폭빨·붕괴 사고…일명 사망 융명 실종
Sentence 5: 반갑다 새해…대전·충남 곳곳서 해맞이 행사
Sentence 6: 최정우 포스코 회장 연임 도전…이사회서 연임 의사 밝혀
Sentence 7: 수능 끝나고 관람오세요 이응노미술관 수험생 무료 혜택
Sentence 8: 화웨이 런정페이 내년까지 스마트폰 美 기술 의존 탈피
Sentence 9: 파국 치닫던 베네수엘라 정부·야권 노르웨이서 대화 모색종합
Sentence 10: 류허 中부총리 방미…10∼11일 워싱턴서 미중 고위급 무역협상종합2보
Sentence 11: 롯데는 왜 살생부 올랐나…검찰 면세점 비리 수사 초점
Sentence 12: 朴대통령 많은 국민 희생후 테러방지법 통과시킨다는건지
Sentence 13: 與 한국당 패스트트랙 수사에 文의장 끌어들여…물귀신 꼼수
Sentence 14: 갤노트9 카메라 화웨이 P20프로에 성능 뒤져
Sentence 15: 양성→4번 음성 광주 학생 2명 가짜 양성…검체 오염 가능성
Sentence 16: 日정부 여론전…주일 외교관 대상 수출규제 설명회 열어
Sentence 17: 한국 남자배구 올림픽 예선서 네덜란드에 뼈아픈 23 역전패
Sentence 18: 애플 韓맞춤형 광고 첫 공개…국내 캐릭터·한글 디자인 활용
Sentence 19: 논문기여도 궁내 이뤼 서울대…세게 육씹파뤼로 이릴게단 하락
Sentence 20: 산학연 협력 연구로 중소기업 연구개발 역량 높인다
Sentence 21: 짐바브웨 전 독재자 무가베 국가영웅묘역에 안장 예정
Sentence 22: 방통위 페이스북 쩝쏙꼉노 변경 조사수위 노펴
Sentence 23: 거대양당 주도 曺청문정국…존재감 약했던 바른미래·정의·평화
Sentence